### selenium
- `https://www.selenium.dev`
- 자동화를 목적으로 만들어진 다양한 브라우져와 언어를 지원하는 라이브러리
- 크롬 브라우져 설치
    - 크롬 브라우져 드라이버 다운로드 (크롬 브라우져와 같은 버전)
        - https://googlechromelabs.github.io/chrome-for-testing/
    - 다운로드한 드라이버 압축 해제
    - chromedriver, chromedriver.exe 생성
    - windows : 주피터 노트북 파일과 동일한 디렉토리에 chromedriver.exe 파일 업로드
    - mac : sudo cp ~/Download/chromedirver /usr/local/bin

In [39]:
# 환경설정
# 1. 크롬 브라우저 설치
# 2. 크롬 브라우저 드라이버 설치 (자동화 위한 다리 역할)
# 3. Selenium 패키지 설치

In [2]:
# !pip install selenium

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/f2/7d/3a0b9c229d87a189b64c3b8e6d87a970a1ef7875995dc31bd18e65fa1c17/selenium-4.22.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/76/51/12d78ec8abcbda51d8f115d98ebd3ee3da9d9d9af00ac69d3097c5b8d51a/trio-0.25.1-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Obtaining dependency information for typing_extensions>=4.9.0 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for websocket-client>=1.8.0 from https://file

'DOSKEY'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [14]:
# selenium 4.0
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

import pandas as pd
import re

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [16]:
driver.get("http://18children.president.pa.go.kr/our_space/fairy_tales.php?srh%5Bcategory%5D=07&srh%5Bpage%5D=1&srh%5Bview_mode%5D=detail&srh%5Bseq%5D=1204")

In [ ]:
# 데이터 수집을 위한 리스트 초기화
title_list = []
collected_descriptions = []
 
# 반복 횟수 제한 설정
max_iterations = 10
iteration = 0
 
while iteration < max_iterations:
    # 타이틀과 내용 수집
    titles = driver.find_elements(By.CSS_SELECTOR, "h3.title1")
    descriptions = driver.find_elements(By.CSS_SELECTOR, ".content")
 
    for title, description in zip(titles, descriptions):
        # 제목에서 'No.' 이후의 부분만 가져오기
        cleaned_title = re.sub(r'^No\.\d+\s+', '', title.text)
        title_list.append(cleaned_title)
        collected_descriptions.append(description.text)
 
        # 제목 출력
        print('제목 :', cleaned_title)
        # 내용 출력
        print('내용 :', description.text)
        # 구분선 출력
        print('---------------------------------------------------------------------------------')
 
    # '다음글' 버튼 클릭
    next_button = driver.find_element(By.CSS_SELECTOR, "a.btn_pack.next")
    driver.execute_script("arguments[0].click();", next_button)
 
    # 다음 페이지가 로드될 때까지 대기 (3초 제한)
    try:
        WebDriverWait(driver, 3).until(EC.staleness_of(next_button))
    except Exception as e:
        print(f"다음 페이지 로딩 대기 중 오류 발생: {e}")
        break
 
    # 반복 횟수 증가
    iteration += 1
 
# 웹 드라이버 종료
driver.quit()
 
# 데이터프레임 생성
df = pd.DataFrame({
    '제목': title_list,
    '내용': collected_descriptions
})
 
# CSV 파일로 저장
df.to_csv('tale_청와대1.csv', index=False, encoding='utf-8-sig')
 
# 수집된 데이터 출력
print("총 수집된 제목 수:", len(title_list))
print("총 수집된 내용 수:", len(collected_descriptions))
print("CSV 파일이 저장되었습니다.")